<a href="https://colab.research.google.com/github/montest/stochastic-methods-optimal-quantization/blob/pytorch_implentation_dim_1/Lloyd_with_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
import os
import sys
import time
import math
import torch
import itertools
import matplotlib

import numpy as np
import pandas as pd
import torch.nn.functional as F

from tqdm import trange

np.set_printoptions(threshold=np.inf, linewidth=10_000)
torch.set_printoptions(profile="full", linewidth=10_000)

### Optimized numpy implementation

In [74]:
def lloyd_method_dim_1(N: int, M: int, nbr_iter: int, seed: int = 0):
    """
    Apply `nbr_iter` iterations of the Randomized Lloyd algorithm in order to build an optimal quantizer of size `N`
    for a Gaussian random variable. This implementation is done using numpy.

    N: number of centroids
    M: number of samples to generate
    nbr_iter: number of iterations of fixed point search
    seed: numpy seed for reproducibility

    Returns: centroids, probabilities associated to each centroid and distortion
    """
    np.random.seed(seed)  # Set seed in order to be able to reproduce the results

    # Draw M samples of gaussian variable
    xs = np.random.normal(0, 1, size=M)

    # Initialize the Voronoi Quantizer randomly and sort it
    centroids = np.random.normal(0, 1, size=N)
    centroids.sort(axis=0)

    with trange(nbr_iter, desc=f'Lloyd method - N: {N} - M: {M} - seed: {seed} (numpy)') as t:
        for step in t:
            # Compute the vertices that separate the centroids
            vertices = 0.5 * (centroids[:-1] + centroids[1:])

            # Find the index of the centroid that is closest to each sample
            index_closest_centroid = np.sum(xs[:, None] >= vertices[None, :], axis=1)

            # Compute the new quantization levels as the mean of the samples assigned to each level
            centroids = np.array([np.mean(xs[index_closest_centroid == i], axis=0) for i in range(N)])

            if any(np.isnan(centroids)):
                break

    # Compute, for each sample, the distance to each centroid
    dist_centroids_points = np.linalg.norm(centroids.reshape((N, 1)) - xs.reshape(M, 1, 1), axis=2)
    # Find the index of the centroid that is closest to each sample using the previously computed distances
    index_closest_centroid = dist_centroids_points.argmin(axis=1)
    # Compute the probability of each centroid
    probabilities = np.bincount(index_closest_centroid) / float(M)
    # Compute the final distortion between the samples and the quantizer
    distortion = np.mean(dist_centroids_points[np.arange(M), index_closest_centroid] ** 2) * 0.5
    return centroids, probabilities, distortion

### PyTorch implementation

In [75]:
def lloyd_method_dim_1_pytorch(N: int, M: int, nbr_iter: int, device: str, seed: int = 0):
    """
    Apply `nbr_iter` iterations of the Randomized Lloyd algorithm in order to build an optimal quantizer of size `N`
    for a Gaussian random variable. This implementation is done using torch.

    N: number of centroids
    M: number of samples to generate
    nbr_iter: number of iterations of fixed point search
    device: device on which perform the computations: "cuda" or "cpu"
    seed: torch seed for reproducibility

    Returns: centroids, probabilities associated to each centroid and distortion
    """
    torch.manual_seed(seed=seed)  # Set seed in order to be able to reproduce the results

    with torch.no_grad():
        # Draw M samples of gaussian variable
        xs = torch.randn(M)
        # xs = torch.tensor(torch.randn(M), dtype=torch.float32)
        xs = xs.to(device)  # send samples to correct device

        # Initialize the Voronoi Quantizer randomly
        centroids = torch.randn(N)
        centroids, index = centroids.sort()
        centroids = centroids.to(device)  # send centroids to correct device

        with trange(nbr_iter, desc=f'Lloyd method - N: {N} - M: {M} - seed: {seed} (pytorch: {device})') as t:
            for step in t:
                # Compute the vertices that separate the centroids
                vertices = 0.5 * (centroids[:-1] + centroids[1:])

                # Find the index of the centroid that is closest to each sample
                index_closest_centroid = torch.sum(xs[:, None] >= vertices[None, :], dim=1).long()

                # Compute the new quantization levels as the mean of the samples assigned to each level
                centroids = torch.tensor([torch.mean(xs[index_closest_centroid == i]) for i in range(N)]).to(device)

                if torch.isnan(centroids).any():
                    break

        # Compute, for each sample, the distance to each centroid
        dist_centroids_points = torch.norm(centroids - xs.reshape(M, 1, 1), dim=1)
        # Find the index of the centroid that is closest to each sample using the previously computed distances
        index_closest_centroid = dist_centroids_points.argmin(dim=1)
        # Compute the probability of each centroid
        probabilities = torch.bincount(index_closest_centroid).to('cpu').numpy()/float(M)
        # Compute the final distortion between the samples and the quantizer
        distortion = torch.mean(dist_centroids_points[torch.arange(M), index_closest_centroid] ** 2).item() * 0.5
        return centroids.to('cpu').numpy(), probabilities, distortion

### Some useful functions for benchmarking

In [78]:
def check_existance(dict_of_values, df):
    v = df.iloc[:, 0] == df.iloc[:, 0]
    for key, value in dict_of_values.items():
        v &= (df[key] == value)
    return v.any()


def testing_method(fct_to_test, parameters_grid: dict, path_to_results: str):
    if os.path.exists(path_to_results) and os.path.getsize(path_to_results) > 0:
        df_results = pd.read_csv(path_to_results, index_col=0)
    else:
        df_results = pd.DataFrame()

    keys, values = zip(*parameters_grid.items())
    permutations_dicts = [dict(zip(keys, v)) for v in itertools.product(*values)]

    for permutations in permutations_dicts:
        dict_result = permutations.copy()
        dict_result["method_name"] = fct_to_test.__name__
        if len(df_results) > 0 and check_existance(dict_result, df_results):
            print(f"Skipping {dict_result}")
            continue

        start_time = time.time()
        centroids, probabilities, distortion = fct_to_test(**permutations)
        if math.isnan(distortion):
            print(f"Results for following values {dict_result} were not saved "
                  f"because an nan was present in the centroids")
            continue
        elapsed_time = time.time() - start_time
        dict_result["elapsed_time"] = elapsed_time
        df_results = pd.concat(
            [df_results, pd.DataFrame(dict_result, index=[0])],
            ignore_index=True
        )
        df_results.to_csv(path_to_results)

### Testing methods

In [79]:
parameters_grid = {
        "N": [10, 20, 50, 100, 200, 500],
        "M": [5000],
        "nbr_iter": [100],
        "seed": [0]
    }
path_to_results = "results_test.csv"

testing_method(lloyd_method_dim_1, parameters_grid, path_to_results)

Lloyd method - N: 50 - M: 5000 - seed: 0 (numpy): 100%|██████████| 100/100 [00:00<00:00, 664.66it/s]
Lloyd method - N: 100 - M: 5000 - seed: 0 (numpy): 100%|██████████| 100/100 [00:00<00:00, 402.56it/s]
Lloyd method - N: 200 - M: 5000 - seed: 0 (numpy): 100%|██████████| 100/100 [00:00<00:00, 217.98it/s]
Lloyd method - N: 500 - M: 5000 - seed: 0 (numpy):   0%|          | 0/100 [00:00<?, ?it/s]/Users/thibautmontes/GitHub/stochastic-methods-optimal-quantization/venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/thibautmontes/GitHub/stochastic-methods-optimal-quantization/venv/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
Lloyd method - N: 500 - M: 5000 - seed: 0 (numpy):   0%|          | 0/100 [00:00<?, ?it/s]

Results for following values {'N': 500, 'M': 5000, 'nbr_iter': 100, 'seed': 0, 'method_name': 'lloyd_method_dim_1'} were not saved because an nan was present in the centroids


In [6]:
if os.path.exists(path_to_results) and os.path.getsize(path_to_results) > 0:
  df_results = pd.read_csv(path_to_results, index_col=0)
df_results['device'].fillna("cpu", inplace=True)
df_results["method"] = df_results.loc[:,"method_name"] + "_" + df_results.loc[:,"device"]
df_results["method"].replace(
  ["lloyd_method_dim_1_cpu", "lloyd_method_dim_1_pytorch_cpu", "lloyd_method_dim_1_pytorch_cuda"],
  ["numpy_cpu", "pytorch_cpu", "pytorch_cuda"],
  inplace=True
  )
df_results["elapsed_time_by_iter"] = df_results.loc[:,"elapsed_time"] / df_results.loc[:,"nbr_iter"]
df_results = df_results[df_results.nbr_iter == 100]
df_results.drop(labels=["method_name", "device", "nbr_iter", "elapsed_time"], axis=1, inplace=True)

In [7]:
df_grouped = df_results.groupby(['N', 'M', 'method'])['elapsed_time_by_iter'].mean()
df_grouped = df_grouped.reset_index()
df_grouped.to_csv("grouped_results.csv")


In [13]:
from bokeh.io import export_svg, export_png
from bokeh.models import ColumnDataSource
from bokeh.palettes import Viridis
from bokeh.plotting import figure, show
import chromedriver_autoinstaller
chromedriver_autoinstaller.install() 

'/Users/thibautmontes/GitHub/stochastic-methods-optimal-quantization/venv/lib/python3.9/site-packages/chromedriver_autoinstaller/110/chromedriver'

In [33]:
toto = df_grouped.groupby(["method", "M"])["elapsed_time_by_iter"].apply(list).to_dict()

def plot_results(M):
    color_numpy_cpu = Viridis[3][1]
    color_pytorch_cpu = Viridis[3][2]
    color_pytorch_cuda = Viridis[3][0]
    general_font_size = '14pt'

    plot = figure(plot_width=600, plot_height=500)

    plot.xaxis.axis_label = "Grid size (N)"
    plot.xaxis.axis_label_text_font_size = general_font_size

    plot.yaxis.axis_label = "Time elapsed per iter (in seconds)"
    plot.yaxis.axis_label_text_font_size = general_font_size

    Ns = df_grouped['N'].unique()
    Ns = sorted(Ns)
    numpy_cpu, pytorch_cpu, pytorch_cuda = toto.get(("numpy_cpu",M)), toto.get(("pytorch_cpu",M)), toto.get(("pytorch_cuda",M))

    source = ColumnDataSource(data=dict(Ns=Ns, numpy_cpu=numpy_cpu, pytorch_cpu=pytorch_cpu, pytorch_cuda=pytorch_cuda))

    plot.circle(x='Ns', y='numpy_cpu', source=source, fill_color=None, line_color=color_numpy_cpu, legend_label='numpy')
    plot.line(x='Ns', y='numpy_cpu', source=source, line_color=color_numpy_cpu, legend_label='numpy')

    plot.circle(x='Ns', y='pytorch_cpu', source=source, fill_color=None, line_color=color_pytorch_cpu, legend_label='pytorch (cpu)')
    plot.line(x='Ns', y='pytorch_cpu', source=source, line_color=color_pytorch_cpu, legend_label='pytorch (cpu)')

    plot.circle(x='Ns', y='pytorch_cuda', source=source, fill_color=color_pytorch_cuda, line_color=color_pytorch_cuda, legend_label='pytorch (cuda)')
    plot.line(x='Ns', y='pytorch_cuda', source=source, line_color=color_pytorch_cuda, legend_label='pytorch (cuda)')

    # show(plot)
    export_png(plot, filename=f"_output/gaussian/pytorch/method_comparison_M_{M}.png")
    export_svg(plot, filename=f"_output/gaussian/pytorch/method_comparison_M_{M}.svg")


In [34]:
plot_results(M=10000)
plot_results(M=20000)
plot_results(M=100000)
plot_results(M=500000)

In [56]:
titi = df_grouped.groupby(["method", "M"])["elapsed_time_by_iter"].apply(np.array).to_dict()
rescaled_comparisons = {(method, M): titi.get((method, M)) / titi.get(("pytorch_cuda", M)) for method, M in titi}
for key, values in rescaled_comparisons.items():
    print(f"{key}: {values}")

('numpy_cpu', 10000): [0.87355883 0.68130139 0.55938944 0.58801913 0.53676187 0.41208587]
('numpy_cpu', 20000): [1.57276189 1.21758264 0.88277627 0.78496016 0.74150523 0.78572709]
('numpy_cpu', 100000): [6.60273571 5.04527401 4.02010785 3.92536147 3.01155591 3.59650244]
('numpy_cpu', 500000): [17.97393747 14.53541768 12.71277519 12.85271106 12.34466679 11.69514007]
('pytorch_cpu', 10000): [1.07585364 1.00480215 0.93223047 0.88571743 0.87278114 0.91807631]
('pytorch_cpu', 20000): [1.79145143 1.69483303 1.59461367 1.59243231 1.68761245 2.43116919]
('pytorch_cpu', 100000): [ 6.9531192   6.73015862  7.34045453 11.54551923  9.37061033 11.17992804]
('pytorch_cpu', 500000): [23.82478392 29.52839536 33.16441265 35.36173497 37.10590004 37.04354136]
('pytorch_cuda', 10000): [1. 1. 1. 1. 1. 1.]
('pytorch_cuda', 20000): [1. 1. 1. 1. 1. 1.]
('pytorch_cuda', 100000): [1. 1. 1. 1. 1. 1.]
('pytorch_cuda', 500000): [1. 1. 1. 1. 1. 1.]


In [62]:
from bokeh.transform import dodge

def plot_ratios(M):
    color_numpy_cpu = Viridis[3][1]
    color_pytorch_cpu = Viridis[3][2]
    color_pytorch_cuda = Viridis[3][0]
    general_font_size = '14pt'

    plot = figure(plot_width=600, plot_height=500)

    plot.xaxis.axis_label = "Grid size (N)"
    plot.xaxis.axis_label_text_font_size = general_font_size

    plot.yaxis.axis_label = "Ratio Time elapsed per iter vs PyTorch cuda"
    plot.yaxis.axis_label_text_font_size = general_font_size

    Ns = sorted(df_grouped['N'].unique())
    methods = ['pytorch_cuda', 'pytorch_cpu', 'numpy_cpu']
    Ns = [str(N) for N in Ns]
    x = [ (N, method) for N in Ns for method in methods ]

    data = {
        'Ns' : Ns,
        'pytorch_cuda' : rescaled_comparisons.get(("pytorch_cuda", M)),
        'pytorch_cpu' : rescaled_comparisons.get(("pytorch_cpu", M)),
        'numpy_cpu' : rescaled_comparisons.get(("numpy_cpu", M))
    }
    
    source = ColumnDataSource(data=data)
    
    plot.vbar(x='Ns', top='pytorch_cuda', source=source, width=0.2, color=color_pytorch_cuda, legend_label="pytorch (cuda)")
    plot.vbar(x='Ns', top='pytorch_cpu', source=source, width=0.2, color=color_pytorch_cpu, legend_label="pytorch (cpu)")
    plot.vbar(x='Ns', top='numpy_cpu', source=source, width=0.2, color=color_numpy_cpu, legend_label="numpy")
    
    # plot.vbar(x=dodge('Ns', -1, range=plot.x_range), top='pytorch_cuda', source=source, width=10, color=color_pytorch_cuda, legend_label="pytorch (cuda)")
    # plot.vbar(x=dodge('Ns',  0.0,  range=plot.x_range), top='pytorch_cpu', source=source, width=10, color=color_pytorch_cpu, legend_label="pytorch (cpu)")
    # plot.vbar(x=dodge('Ns',  1, range=plot.x_range), top='numpy_cpu', source=source, width=10, color=color_numpy_cpu, legend_label="numpy")

    # show(plot)
    export_png(plot, filename=f"_output/gaussian/pytorch/ratio_comparison_M_{M}.png")
    export_svg(plot, filename=f"_output/gaussian/pytorch/ratio_comparison_M_{M}.svg")

In [63]:
plot_ratios(M=10000)
# plot_ratios(M=20000)
# plot_ratios(M=100000)
# plot_ratios(M=500000)